In [13]:
import numpy as np
import numpy.ma as ma
import pandas as pd

In [14]:
class SimilarityMetric:

    def calculateSimilarity(self, u, v):
        pass

In [15]:
class CosineSimilarity(SimilarityMetric):
    def calculateSimilarity(self, u, v):
        u = np.nan_to_num(u, nan=0)
        v = np.nan_to_num(v, nan=0)
        return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v) + 1e-9)

class PCCSimilarity(SimilarityMetric):
    def calculateSimilarity(self, u, v):
        return self.__pearson_correlation(u, v)
    
    def __pearson_correlation(self, user1_ratings, user2_ratings):

        x_masked = ma.masked_invalid(user1_ratings)
        y_masked = ma.masked_invalid(user2_ratings)

        # Create a mask where both x_masked and y_masked are null
        combined_mask = ~x_masked.mask & ~y_masked.mask

        # Find the indices where both x_masked and y_masked are null
        non_null_indices = np.where(combined_mask)
                
        if len(non_null_indices) < 2:  # Require at least 2 common items for correlation
            return 0
        
        correlation_matrix = ma.corrcoef(x_masked, y_masked)
        correlation = correlation_matrix[0,1]

        return correlation

In [16]:
class CollaborativeFiltering:

    def __init__(self, data, metric:SimilarityMetric):
        """
        Initialize CollaborativeFiltering object with user-item rating data.
        
        Parameters:
        - data: DataFrame containing user-item ratings
        """
        self.data = data
        self.train_data = None
        self.test_data = None
        self.similarity_matrix = None
        self.metric = metric
    
    def calculate_similarity_matrix(self):
        pass

    def predict_ratings(self, user_id):
        pass

    def train_test_split(self, test_size = 0.2):
        """
        Split the data into training and test sets.
        
        Parameters:
        - test_size: Fraction of the data to be used for testing
        """
        np.random.seed(42)  # for reproducibility
        mask = np.random.rand(len(self.data)) < 1 - test_size
        self.train_data = self.data[mask]
        self.test_data = self.data[~mask]

    def evaluate(self):
        """
        Evaluate the Collaborative Filtering model on the test set.

        Returns:
        - Mean squared error (MSE) of the predictions
        """

        mse_sum = 0
        total_predictions = 0

        for user_id in self.test_data.index:
            user_test_ratings = self.test_data.loc[user_id]
            user_predicted_ratings = self.predict_ratings(user_id)
            for movie_id, actual_rating in user_test_ratings.items():
                predicted_rating = user_predicted_ratings.loc[movie_id, 'PredictedRating']
                if not pd.isnull(predicted_rating):
                    total_predictions += 1
                    mse_sum += (actual_rating - predicted_rating) ** 2

        mse = mse_sum / total_predictions
        return mse

In [17]:
class CollaborativeFilteringItemItem(CollaborativeFiltering):

    def __init__(self, data, metric:SimilarityMetric):
        super().__init__(data, metric)
    

    def calculate_similarity_matrix(self):
        
        # user_item_matrix = self.train_data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)
        
        n_movies = self.data.shape[1]
        similarity_matrix = np.zeros((n_movies, n_movies))
        for i in range(n_movies):
            for j in range(i+1):
                temp = self.metric.calculateSimilarity(self.data.iloc[:, i], self.data.iloc[:, j])
                similarity_matrix[i, j] = temp
                similarity_matrix[j, i] = temp

        self.similarity_matrix = pd.DataFrame(similarity_matrix, index=self.data.columns, columns=self.data.columns)

    def predict_ratings(self, user_id):
        """
        Predict ratings for items for a given user.

        Parameters:
        - user_id: ID of the user for whom to predict ratings

        Returns:
        - DataFrame containing predicted ratings for each item
        """

        user_ratings = self.data.loc[user_id]
        predicted_ratings = pd.DataFrame(index=self.data.columns, columns=['PredictedRating'])

        for movie_id in predicted_ratings.index:
            numerator = 0
            denominator = 0

            for other_movie_id in predicted_ratings.index:
                if (other_movie_id != movie_id):
                    similarity = self.similarity_matrix.loc[movie_id, other_movie_id]
                    other_movie_rating = user_ratings[other_movie_id]

                    if not np.isnan(other_movie_rating):
                        numerator += similarity * other_movie_rating
                        denominator += abs(similarity)
            
            predicted_ratings.loc[movie_id, 'PredictedRating'] = numerator / (denominator + 1e-9)
            
        return predicted_ratings
        

In [18]:
class CollaborativeFilteringUserUser(CollaborativeFiltering):
    def __init__(self, data, metric:SimilarityMetric):
        
        super().__init__(data, metric)

    def calculate_similarity_matrix(self):
        
        # user_item_matrix = self.train_data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)

        n_users = self.data.shape[0]
        similarity_matrix = np.zeros((n_users, n_users))
        for i in range(n_users):
            for j in range(i+1):
                temp = self.metric.calculateSimilarity(self.data.iloc[i, :], self.data.iloc[j, :])
                similarity_matrix[i, j] = temp
                similarity_matrix[j, i] = temp
                
        self.similarity_matrix = pd.DataFrame(similarity_matrix, index=self.data.index, columns=self.data.index)

    def predict_ratings(self, user_id):
        """
        Predict ratings for items for a given user.

        Parameters:
        - user_id: ID of the user for whom to predict ratings

        Returns:
        - DataFrame containing predicted ratings for each item
        """

        predicted_ratings = pd.DataFrame(index=self.data.columns, columns=['PredictedRating'])

        for movie_id in predicted_ratings.index:
            numerator = 0
            denominator = 0

            for other_user_id in self.data.index:
                other_user_rating = self.data.loc[other_user_id, movie_id]
                similarity = self.similarity_matrix.loc[user_id , other_user_id]

                if not pd.isnull(other_user_rating):
                    numerator += similarity * (other_user_rating)
                    denominator += abs(similarity)

            predicted_ratings.loc[movie_id, 'PredictedRating'] = numerator / (denominator + 1e-9) 
            
        return predicted_ratings

In [19]:
data=pd.read_csv("EncodedCombined.csv")

user_item_matrix = data.pivot(index='UserID', columns='MovieID', values='Rating')
user_item_matrix_new = user_item_matrix

/tmp/ipykernel_222198/103287248.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("EncodedCombined.csv")


In [20]:
# metric = CosineSimilarity()
# # Create CollaborativeFiltering instance
# cf = CollaborativeFilteringItemItem(user_item_matrix_new, metric)

# # Split data into train and test sets
# cf.train_test_split(test_size=0.2)
# cf.calculate_similarity_matrix()

# # Evaluate the model
# mse = cf.evaluate()
# print("Mean Squared Error:", mse)

In [21]:
# prediction_df = cf.predict_ratings(2)
# print(prediction_df.sort_values(by=['PredictedRating']).head(10))

In [22]:
# metric = PCCSimilarity()
# cf = CollaborativeFilteringItemItem(user_item_matrix_new, metric)

# # Split data into train and test sets
# cf.train_test_split(test_size=0.2)
# cf.calculate_similarity_matrix()

# # Evaluate the model
# mse = cf.evaluate()
# print("Mean Squared Error:", mse)

In [23]:
# prediction_df = cf.predict_ratings(2)
# print(prediction_df.sort_values(by=['PredictedRating']).head(10))

In [24]:
metric = CosineSimilarity()
# Create CollaborativeFiltering instance
cf = CollaborativeFilteringUserUser(user_item_matrix_new, metric)

# Split data into train and test sets
cf.train_test_split(test_size=0.2)
cf.calculate_similarity_matrix()

# Evaluate the model
# mse = cf.evaluate()
# print("Mean Squared Error:", mse)

In [25]:
prediction_df = cf.predict_ratings(2)
print(prediction_df.sort_values(by=['PredictedRating']).tail(10))

        PredictedRating
MovieID                
3382           4.999999
1830                5.0
2480                5.0
3656                5.0
989                 5.0
3881                5.0
3607                5.0
3172                5.0
3233                5.0
787                 5.0


In [26]:
metric = PCCSimilarity()
cf = CollaborativeFilteringUserUser(user_item_matrix_new, metric)

# Split data into train and test sets
cf.train_test_split(test_size=0.2)
cf.calculate_similarity_matrix()

# Evaluate the model
# mse = cf.evaluate()
# print("Mean Squared Error:", mse)

In [ ]:
prediction_df = cf.predict_ratings(2)
print(prediction_df.sort_values(by=['PredictedRating']).tail(10))